In [5]:

#Basic py: 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import math
import json
from urllib.request import urlopen
import glob
#Vis
#import seaborn as sns

#Geo
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 
import xarray as xr 




In [2]:
def nc_open(path):
    """
    A function to open netcdf4 files. Requires xarray
    ===
    Input:
    path - path to the cdf file

    Output: 
    df - converted cdf file to dataframe object
    """
    df = xr.open_dataarray(path)
    return df.to_dataframe().reset_index()

In [3]:
def file_to_geoframe(path):
    """A function to take a shapefile with concentration element and convert to a geodataframe
    """
    gdf = gpd.read_file(path)
    gdf = gpd.GeoDataFrame(gdf,crs=gdf.crs)

    try:
        gdf['CONC']=gdf['CONC'].astype(float)
    except:
        print('No concentration data')

    gdf = gdf.to_crs('epsg:3857')
    return gdf

def stilt_post_processing(stilt_df, threshold):
    '''Takes a stilt particle output, in a pandas dataframe, and converts to a geopandas object.  

    Input:
    ----------
    stilt_df - an output coming from nc_open, based upon netcdf to pandas conversion
    threshold - a value for filtering - if null no filtering is performed on the data. 

    Returns:
    sim_avg: a geodataframe of the average non-log_conc per the simulation run (48 hr with current setup) transformed to points for comparison
    '''  
    if threshold != None:
        stilt_df = stilt_df[stilt_df.foot>threshold] 
    
    sim_avg = stilt_df.groupby(['lat','lon']).agg({'foot':'mean'}).reset_index()

    #Converting to a geo-type object
    sim_avg = gpd.GeoDataFrame(sim_avg, geometry=gpd.points_from_xy(sim_avg.lon, sim_avg.lat)).set_crs(epsg=4326)
    sim_avg = sim_avg.to_crs(epsg=3857)
    return sim_avg

In [6]:
stilt_11189_4071 = nc_open('/home/boogie2/Hanson_Lab/TRI_STILT/data/raw/STILT/090420_base_run/199001010000_-111.89273_40.70739_0_foot.nc')
stilt_11189_4071_sim_avg = stilt_post_processing(stilt_11189_4071,threshold=0.001)

In [7]:
stilt_11189_4071_sim_avg

,lat,lon,foot,geometry
0,40.705,-111.895,0.171536,POINT (-12456094.422 4968926.125)
1,40.715,-111.895,0.189736,POINT (-12456094.422 4970394.680)
2,40.725,-111.915,0.001140,POINT (-12458320.812 4971863.456)
3,40.725,-111.905,0.001140,POINT (-12457207.617 4971863.456)
4,40.725,-111.895,0.068439,POINT (-12456094.422 4971863.456)
...,...,...,...,...
507,42.015,-109.005,0.003177,POINT (-12134381.094 5163226.636)
508,42.025,-109.005,0.003492,POINT (-12134381.094 5164725.059)
509,42.035,-109.005,0.003779,POINT (-12134381.094 5166223.717)
510,42.045,-109.005,0.004028,POINT (-12134381.094 5167722.611)
